***GENERATED CODE FOR workethicandproductivitycorrelationregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'projectname', 'transformation_label': 'String Indexer'}], 'feature': 'projectname', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': ' AI Forge: Pathway to Predictive Excellence', 'max': 'The Effect of Holiday Sales on Walmart Using ML', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'projectname'}, {'feature_label': 'projectname', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('projectname')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProjectStartDate', 'transformation_label': 'Extract Date'}], 'feature': 'ProjectStartDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ProjectStartDate'}, {'feature_label': 'ProjectStartDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ProjectStartDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern', 'transformation_label': 'String Indexer'}], 'feature': 'Intern', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Abel Kirui', 'max': 'Zaryab Prasla', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern'}, {'feature_label': 'Intern', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'intern_mm_weekenddate', 'transformation_label': 'Extract Date'}], 'feature': 'intern_mm_weekenddate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '1'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'intern_mm_weekenddate'}, {'feature_label': 'intern_mm_weekenddate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('intern_mm_weekenddate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FirstName', 'transformation_label': 'String Indexer'}], 'feature': 'FirstName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ANAMELECHI KINGSLEY', 'max': 'Zaryab', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FirstName'}, {'feature_label': 'FirstName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FirstName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LastName', 'transformation_label': 'String Indexer'}], 'feature': 'LastName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Gebreegziabher', 'max': 'waites', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LastName'}, {'feature_label': 'LastName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LastName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Email', 'transformation_label': 'String Indexer'}], 'feature': 'Email', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Etse2004@yahoo.com', 'max': 'zaryabprasla35@gmail.com', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Email'}, {'feature_label': 'Email', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Email')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReportDate', 'transformation_label': 'Extract Date'}], 'feature': 'ReportDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ReportDate'}, {'feature_label': 'ReportDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ReportDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopDISCscore', 'transformation_label': 'String Indexer'}], 'feature': 'TopDISCscore', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Compliance', 'max': 'Steadiness', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopDISCscore'}, {'feature_label': 'TopDISCscore', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopDISCscore')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LowDISCscore', 'transformation_label': 'String Indexer'}], 'feature': 'LowDISCscore', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Dominance', 'max': 'Steadiness', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LowDISCscore'}, {'feature_label': 'LowDISCscore', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LowDISCscore')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopMotivator', 'transformation_label': 'String Indexer'}], 'feature': 'TopMotivator', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aesthetic', 'max': 'Utilitarian', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopMotivator'}, {'feature_label': 'TopMotivator', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopMotivator')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LowMotivator', 'transformation_label': 'String Indexer'}], 'feature': 'LowMotivator', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aesthetic', 'max': 'Utilitarian', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LowMotivator'}, {'feature_label': 'LowMotivator', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LowMotivator')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopSkill', 'transformation_label': 'String Indexer'}], 'feature': 'TopSkill', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Flexibility', 'max': 'Teamwork', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopSkill'}, {'feature_label': 'TopSkill', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopSkill')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LowSkill', 'transformation_label': 'String Indexer'}], 'feature': 'LowSkill', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'FuturisticThinking', 'max': 'Presenting', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LowSkill'}, {'feature_label': 'LowSkill', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LowSkill')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'GoalResponseLen', 'transformation_label': 'String Indexer'}], 'feature': 'GoalResponseLen', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'GoalResponseLen'}, {'feature_label': 'GoalResponseLen', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('GoalResponseLen')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternEmail', 'transformation_label': 'String Indexer'}], 'feature': 'InternEmail', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'amirwaites@gmail.com', 'max': 'zaryabprasla35@gmail.com', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternEmail'}, {'feature_label': 'InternEmail', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternEmail')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternStartDate', 'transformation_label': 'Extract Date'}], 'feature': 'InternStartDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'InternStartDate'}, {'feature_label': 'InternStartDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('InternStartDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternEndDate', 'transformation_label': 'String Indexer'}], 'feature': 'InternEndDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternEndDate'}, {'feature_label': 'InternEndDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternEndDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternTechGroupName', 'transformation_label': 'String Indexer'}], 'feature': 'InternTechGroupName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'PowerBI', 'max': 'PowerBI', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternTechGroupName'}, {'feature_label': 'InternTechGroupName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternTechGroupName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternManager', 'transformation_label': 'String Indexer'}], 'feature': 'InternManager', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Ali Muwwakkil', 'max': 'Ali Muwwakkil', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternManager'}, {'feature_label': 'InternManager', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternManager')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternJobLocation', 'transformation_label': 'String Indexer'}], 'feature': 'InternJobLocation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Anywhere, US', 'max': 'DFW, TX', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternJobLocation'}, {'feature_label': 'InternJobLocation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternJobLocation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternBaseCampAlias', 'transformation_label': 'String Indexer'}], 'feature': 'InternBaseCampAlias', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '36', 'mean': '', 'stddev': '', 'min': 'OBI, ANAMELECHI KINGSLEY', 'max': 'OBI, ANAMELECHI KINGSLEY', 'missing': '464', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternBaseCampAlias'}, {'feature_label': 'InternBaseCampAlias', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternBaseCampAlias')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PName', 'transformation_label': 'String Indexer'}], 'feature': 'PName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' AI Forge: Pathway to Predictive Excellence', 'max': 'The Effect of Holiday Sales on Walmart Using ML', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PName'}, {'feature_label': 'PName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'L_URL', 'transformation_label': 'String Indexer'}], 'feature': 'L_URL', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'https://3.basecamp.com/3945211/buckets/24865175/todolists/6755822834', 'max': 'https://3.basecamp.com/3945211/buckets/24865175/todolists/7255296952', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'L_URL'}, {'feature_label': 'L_URL', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('L_URL')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_LastActivitySection', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_LastActivitySection', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '499', 'mean': '', 'stddev': '', 'min': 'Phase 1 - Automated Mock - Mary Gathogo_02/05/2024_CCM3_Data Architect',
                                                                                                                                                                                                                                                                                                      'max': 'Phase 3 - Automated Mock - Yannick Ntwari_04/03/2024_Bottomline Technologies_Senior Data Analyst', 'missing': '1', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_LastActivitySectio...'}, {'feature_label': 'Intern_LastActivitySection', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Intern_LastActivitySection')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_MM_Mentor', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_MM_Mentor', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '', 'stddev': '', 'min': 'ALEXANDER  OPIYO', 'max': 'Shireesha Ravi', 'missing': '1', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_MM_Mentor'}, {'feature_label': 'Intern_MM_Mentor', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern_MM_Mentor')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_MM_PlacementDate', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_MM_PlacementDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_MM_PlacementDate'}, {'feature_label': 'Intern_MM_PlacementDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern_MM_PlacementDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_MM_WklyComments', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_MM_WklyComments', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_MM_WklyComments'}, {'feature_label': 'Intern_MM_WklyComments', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern_MM_WklyComments')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_MM_LastCommentDate', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_MM_LastCommentDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_MM_LastCommentDate'}, {'feature_label': 'Intern_MM_LastCommentDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Intern_MM_LastCommentDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Phase', 'transformation_label': 'String Indexer'}], 'feature': 'Phase', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Phase 1', 'max': 'Phase 3', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Phase'}, {'feature_label': 'Phase', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Phase')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CurrentSprintTask', 'transformation_label': 'String Indexer'}], 'feature': 'CurrentSprintTask', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CurrentSprintTask'}, {'feature_label': 'CurrentSprintTask', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CurrentSprintTask')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StudentProfile', 'transformation_label': 'String Indexer'}], 'feature': 'StudentProfile', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'https://app.colaberry.com/app/ipbc/students/ipbc/31790', 'max': 'https://app.colaberry.com/app/ipbc/students/ipbc/40312', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StudentProfile'}, {'feature_label': 'StudentProfile', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StudentProfile')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProjectName_2', 'transformation_label': 'String Indexer'}], 'feature': 'ProjectName_2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' AI Forge: Pathway to Predictive Excellence', 'max': 'The Effect of Holiday Sales on Walmart Using ML', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProjectName_2'}, {'feature_label': 'ProjectName_2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProjectName_2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProjectKPI_Link', 'transformation_label': 'String Indexer'}], 'feature': 'ProjectKPI_Link', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'https://app.colaberry.com/app/student/ProjMgmtByAdmin/1778', 'max': 'https://app.colaberry.com/app/student/ProjMgmtByAdmin/1956', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProjectKPI_Link'}, {'feature_label': 'ProjectKPI_Link', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProjectKPI_Link')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run workethicandproductivitycorrelationregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	ai = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/AI.csv', 'filename': 'AI.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(ai)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run workethicandproductivitycorrelationregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	workethicandproductivitycorrelationregressionautofe = TransformationMain.run(ai,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "projectname", "transformation_label": "String Indexer"}], "feature": "projectname", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " AI Forge: Pathway to Predictive Excellence", "max": "The Effect of Holiday Sales on Walmart Using ML", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "projectname"}, {"transformationsData": [{"feature_label": "ProjectStartDate", "transformation_label": "Extract Date"}], "feature": "ProjectStartDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ProjectStartDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "internship_score2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.63", "stddev": "0.19", "min": "0.25", "max": "0.922222", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "internship_score2"}, {"transformationsData": [{"feature_label": "Intern", "transformation_label": "String Indexer"}], "feature": "Intern", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Abel Kirui", "max": "Zaryab Prasla", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternUserID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "36740.36", "stddev": "2488.47", "min": "31790", "max": "40312", "missing": "0"}, "updatedLabel": "InternUserID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "intern_mm_wkorder5", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "5.15", "stddev": "3.63", "min": "1.0", "max": "12.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "intern_mm_wkorder5"}, {"transformationsData": [{"feature_label": "intern_mm_weekenddate", "transformation_label": "Extract Date"}], "feature": "intern_mm_weekenddate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "1"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "intern_mm_weekenddate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WkScore", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.38", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "WkScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "UserID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "36740.36", "stddev": "2488.47", "min": "31790", "max": "40312", "missing": "0"}, "updatedLabel": "UserID"}, {"transformationsData": [{"feature_label": "FirstName", "transformation_label": "String Indexer"}], "feature": "FirstName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ANAMELECHI KINGSLEY", "max": "Zaryab", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FirstName"}, {"transformationsData": [{"feature_label": "LastName", "transformation_label": "String Indexer"}], "feature": "LastName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Gebreegziabher", "max": "waites", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LastName"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Dominance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "39.99", "stddev": "27.12", "min": "7", "max": "89", "missing": "0"}, "updatedLabel": "Dominance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Influencer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "50.38", "stddev": "12.38", "min": "8", "max": "64", "missing": "0"}, "updatedLabel": "Influencer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Steadiness", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "61.89", "stddev": "26.18", "min": "18", "max": "94", "missing": "0"}, "updatedLabel": "Steadiness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Compliance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "63.54", "stddev": "19.12", "min": "38", "max": "92", "missing": "0"}, "updatedLabel": "Compliance"}, {"transformationsData": [{"feature_label": "Email", "transformation_label": "String Indexer"}], "feature": "Email", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Etse2004@yahoo.com", "max": "zaryabprasla35@gmail.com", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Email"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Theoretical", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "57.09", "stddev": "10.03", "min": "40", "max": "78", "missing": "0"}, "updatedLabel": "Theoretical"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Utilitarian", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "50.47", "stddev": "9.15", "min": "27", "max": "73", "missing": "0"}, "updatedLabel": "Utilitarian"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Aesthetic", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "43.85", "stddev": "8.76", "min": "31", "max": "59", "missing": "0"}, "updatedLabel": "Aesthetic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Social", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "41.03", "stddev": "21.93", "min": "8", "max": "70", "missing": "0"}, "updatedLabel": "Social"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Individualistic", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "43.02", "stddev": "6.03", "min": "26", "max": "57", "missing": "0"}, "updatedLabel": "Individualistic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Traditional", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "33.81", "stddev": "11.57", "min": "18", "max": "65", "missing": "0"}, "updatedLabel": "Traditional"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Flexibility", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "31.04", "stddev": "31.31", "min": "2.3", "max": "79.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Flexibility"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FuturisticThinking", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "4.24", "stddev": "10.9", "min": "0.0", "max": "43.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "FuturisticThinking"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "GoalOrientation", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "21.64", "stddev": "25.91", "min": "4.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "GoalOrientation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InterpersonalSkills", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "29.38", "stddev": "32.13", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InterpersonalSkills"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Leadership", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "26.84", "stddev": "30.59", "min": "0.0", "max": "83.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Leadership"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Negotiation", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "20.16", "stddev": "26.85", "min": "0.3", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Negotiation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PersonalResponsibility", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "29.49", "stddev": "30.46", "min": "4.1", "max": "81.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PersonalResponsibility"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Persuasion", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.94", "stddev": "21.02", "min": "0.0", "max": "83.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Persuasion"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PlanningOrganizing", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "30.08", "stddev": "33.64", "min": "2.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PlanningOrganizing"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Presenting", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "18.98", "stddev": "26.18", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Presenting"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Teamwork", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "37.38", "stddev": "40.34", "min": "4.3", "max": "93.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Teamwork"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WrittenCommunication", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "17.95", "stddev": "24.14", "min": "0.0", "max": "90.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "WrittenCommunication"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "UnderstandingOthers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "28.83", "stddev": "28.79", "min": "0.0", "max": "78.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "UnderstandingOthers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PracticalThinking", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "34.53", "stddev": "36.03", "min": "2.2", "max": "84.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PracticalThinking"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SystemsJudgment", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "29.2", "stddev": "28.44", "min": "4.8", "max": "78.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SystemsJudgment"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SenseOfSelf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "32.41", "stddev": "35.31", "min": "3.0", "max": "82.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SenseOfSelf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RoleAwareness", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "33.14", "stddev": "36.47", "min": "1.0", "max": "88.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RoleAwareness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SelfDirection", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "26.49", "stddev": "26.75", "min": "3.6", "max": "76.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SelfDirection"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "UnderstandingOthersBias", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "-0.97", "stddev": "0.18", "min": "-1", "max": "1", "missing": "0"}, "updatedLabel": "UnderstandingOthersBias"}, {"transformationsData": [{"feature_label": "ReportDate", "transformation_label": "Extract Date"}], "feature": "ReportDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ReportDate"}, {"transformationsData": [{"feature_label": "TopDISCscore", "transformation_label": "String Indexer"}], "feature": "TopDISCscore", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Compliance", "max": "Steadiness", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopDISCscore"}, {"transformationsData": [{"feature_label": "LowDISCscore", "transformation_label": "String Indexer"}], "feature": "LowDISCscore", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Dominance", "max": "Steadiness", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LowDISCscore"}, {"transformationsData": [{"feature_label": "TopMotivator", "transformation_label": "String Indexer"}], "feature": "TopMotivator", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aesthetic", "max": "Utilitarian", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopMotivator"}, {"transformationsData": [{"feature_label": "LowMotivator", "transformation_label": "String Indexer"}], "feature": "LowMotivator", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aesthetic", "max": "Utilitarian", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LowMotivator"}, {"transformationsData": [{"feature_label": "TopSkill", "transformation_label": "String Indexer"}], "feature": "TopSkill", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Flexibility", "max": "Teamwork", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopSkill"}, {"transformationsData": [{"feature_label": "LowSkill", "transformation_label": "String Indexer"}], "feature": "LowSkill", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "FuturisticThinking", "max": "Presenting", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LowSkill"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ClassSignupsID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "13458.68", "stddev": "1171.29", "min": "9575", "max": "15244", "missing": "0"}, "updatedLabel": "ClassSignupsID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveySent", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.55", "stddev": "0.74", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "SurveySent"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StagnantWks", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "44", "mean": "1.77", "stddev": "1.29", "min": "0.0", "max": "3.0", "missing": "456"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StagnantWks"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INTERNSHIP_SCORE52", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.6", "stddev": "0.2", "min": "0.25", "max": "0.922222", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INTERNSHIP_SCORE52"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveysTaken", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "3.02", "stddev": "9.53", "min": "0", "max": "54", "missing": "0"}, "updatedLabel": "SurveysTaken"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveysGiven", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.36", "stddev": "6.51", "min": "0", "max": "37", "missing": "0"}, "updatedLabel": "SurveysGiven"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "QuestionsScored", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "30.57", "stddev": "84.5", "min": "0", "max": "481", "missing": "0"}, "updatedLabel": "QuestionsScored"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Work Ethic", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.6", "stddev": "0.44", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Work Ethic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Availability", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.63", "stddev": "0.46", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Availability"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Team Player", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.62", "stddev": "0.45", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Team Player"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Quality of Work", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.62", "stddev": "0.45", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Quality of Work"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Trust_2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.62", "stddev": "0.45", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Trust_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Respectful", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.63", "stddev": "0.46", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Respectful"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveyScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.62", "stddev": "0.45", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SurveyScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCamp_FinalScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.47", "stddev": "0.41", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCamp_FinalScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_FinalScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.42", "stddev": "0.31", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_FinalScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCamp_Score", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.47", "stddev": "0.41", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCamp_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Attendance", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "297", "mean": "2.38", "stddev": "1.17", "min": "1.0", "max": "5.0", "missing": "203"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Attendance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCampCommentScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "297", "mean": "2.38", "stddev": "1.17", "min": "1.0", "max": "5.0", "missing": "203"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCampCommentScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCampTotalComments", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "297", "mean": "4.26", "stddev": "3.1", "min": "1.0", "max": "17.0", "missing": "203"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCampTotalComments"}, {"transformationsData": [{"feature_label": "GoalResponseLen", "transformation_label": "String Indexer"}], "feature": "GoalResponseLen", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "GoalResponseLen"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Score", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.42", "stddev": "0.31", "min": "0.0", "max": "1.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "423.46", "stddev": "18.47", "min": "330", "max": "445", "missing": "0"}, "updatedLabel": "InternID"}, {"transformationsData": [{"feature_label": "InternEmail", "transformation_label": "String Indexer"}], "feature": "InternEmail", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "amirwaites@gmail.com", "max": "zaryabprasla35@gmail.com", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternEmail"}, {"transformationsData": [{"feature_label": "InternStartDate", "transformation_label": "Extract Date"}], "feature": "InternStartDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "InternStartDate"}, {"transformationsData": [{"feature_label": "InternEndDate", "transformation_label": "String Indexer"}], "feature": "InternEndDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternEndDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternTechGroupID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "3.0", "stddev": "0.0", "min": "3", "max": "3", "missing": "0"}, "updatedLabel": "InternTechGroupID"}, {"transformationsData": [{"feature_label": "InternTechGroupName", "transformation_label": "String Indexer"}], "feature": "InternTechGroupName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "PowerBI", "max": "PowerBI", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternTechGroupName"}, {"transformationsData": [{"feature_label": "InternManager", "transformation_label": "String Indexer"}], "feature": "InternManager", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Ali Muwwakkil", "max": "Ali Muwwakkil", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternManager"}, {"transformationsData": [{"feature_label": "InternJobLocation", "transformation_label": "String Indexer"}], "feature": "InternJobLocation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Anywhere, US", "max": "DFW, TX", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternJobLocation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "internisactive", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "internisactive"}, {"transformationsData": [{"feature_label": "InternBaseCampAlias", "transformation_label": "String Indexer"}], "feature": "InternBaseCampAlias", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "36", "mean": "", "stddev": "", "min": "OBI, ANAMELECHI KINGSLEY", "max": "OBI, ANAMELECHI KINGSLEY", "missing": "464", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternBaseCampAlias"}, {"transformationsData": [{"feature_label": "PName", "transformation_label": "String Indexer"}], "feature": "PName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " AI Forge: Pathway to Predictive Excellence", "max": "The Effect of Holiday Sales on Walmart Using ML", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PName"}, {"transformationsData": [{"feature_label": "L_URL", "transformation_label": "String Indexer"}], "feature": "L_URL", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "https://3.basecamp.com/3945211/buckets/24865175/todolists/6755822834", "max": "https://3.basecamp.com/3945211/buckets/24865175/todolists/7255296952", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "L_URL"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_WkOrder83", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6.49", "stddev": "3.49", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "Intern_MM_WkOrder83"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Wk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "10.5", "stddev": "3.48", "min": "5.0", "max": "16.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Wk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Yr", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "2024.0", "stddev": "0.0", "min": "2024.0", "max": "2024.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Yr"}, {"transformationsData": [{"feature_label": "Intern_LastActivitySection", "transformation_label": "String Indexer"}], "feature": "Intern_LastActivitySection", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "Phase 1 - Automated Mock - Mary Gathogo_02/05/2024_CCM3_Data Architect", "max": "Phase 3 - Automated Mock - Yannick Ntwari_04/03/2024_Bottomline Technologies_Senior Data Analyst", "missing": "1", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_LastActivitySectio..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_HW", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_HW"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Videos", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.29", "stddev": "0.56", "min": "0.0", "max": "2.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Videos"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_VideosScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "1.02", "stddev": "1.87", "min": "0.0", "max": "5.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_VideosScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_AutoInt", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.62", "stddev": "1.27", "min": "0.0", "max": "7.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_AutoInt"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Phone", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.12", "stddev": "0.39", "min": "0.0", "max": "3.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Phone"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_LoginPer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.59", "stddev": "0.31", "min": "0.0", "max": "1.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_LoginPer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ActivityScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "1.02", "stddev": "1.61", "min": "0.0", "max": "8.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_ActivityScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ParticipationPer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.4", "stddev": "0.49", "min": "0.0", "max": "1.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_ParticipationPe..."}, {"transformationsData": [{"feature_label": "Intern_MM_Mentor", "transformation_label": "String Indexer"}], "feature": "Intern_MM_Mentor", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "ALEXANDER  OPIYO", "max": "Shireesha Ravi", "missing": "1", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Mentor"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_UserComments", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.86", "stddev": "0.84", "min": "0.0", "max": "4.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_UserComments"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ActivityScoreChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.4", "stddev": "0.49", "min": "0.0", "max": "1.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_ActivityScoreCh..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_UserCommentsChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.66", "stddev": "0.47", "min": "0.0", "max": "1.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_UserCommentsChk"}, {"transformationsData": [{"feature_label": "Intern_MM_PlacementDate", "transformation_label": "String Indexer"}], "feature": "Intern_MM_PlacementDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_PlacementDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_JRPChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_JRPChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_SubChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_SubChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_AutoIntChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.28", "stddev": "0.45", "min": "0.0", "max": "1.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_AutoIntChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_PhoneChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.09", "stddev": "0.29", "min": "0.0", "max": "1.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_PhoneChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Wk_ActivityScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.24", "stddev": "0.13", "min": "0.11", "max": "0.588571", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Wk_ActivityScor..."}, {"transformationsData": [{"feature_label": "Intern_MM_WklyComments", "transformation_label": "String Indexer"}], "feature": "Intern_MM_WklyComments", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_WklyComments"}, {"transformationsData": [{"feature_label": "Intern_MM_LastCommentDate", "transformation_label": "String Indexer"}], "feature": "Intern_MM_LastCommentDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_LastCommentDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_Answer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "386", "mean": "1.67", "stddev": "4.85", "min": "0.0", "max": "28.0", "missing": "114"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_CC_Answer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_CorrectAnswer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "386", "mean": "0.01", "stddev": "0.09", "min": "0.0", "max": "1.0", "missing": "114"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_CC_CorrectAnswer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_VideosHosted", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "386", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "114"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_CC_VideosHosted"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_QualfyVideo", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "386", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "114"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_CC_QualfyVideo"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_COE_Participation", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "386", "mean": "6.25", "stddev": "3.8", "min": "1.0", "max": "18.0", "missing": "114"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_COE_Participation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_COE_ProjectUpdates", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "386", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "114"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_COE_ProjectUpdates"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "1.02", "stddev": "1.61", "min": "0.0", "max": "8.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Part1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.89", "stddev": "2.34", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "IPBC_Part2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4.5", "stddev": "3.63", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "IPBC_Part3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Score_Pre_Total", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "6.42", "stddev": "5.03", "min": "0.0", "max": "20.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Score_Pre_Total"}, {"transformationsData": [{"feature_label": "Phase", "transformation_label": "String Indexer"}], "feature": "Phase", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Phase 1", "max": "Phase 3", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Phase"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternLevelGroup", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.1", "stddev": "0.34", "min": "2.0", "max": "4.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InternLevelGroup"}, {"transformationsData": [{"feature_label": "CurrentSprintTask", "transformation_label": "String Indexer"}], "feature": "CurrentSprintTask", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CurrentSprintTask"}, {"transformationsData": [{"feature_label": "StudentProfile", "transformation_label": "String Indexer"}], "feature": "StudentProfile", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "https://app.colaberry.com/app/ipbc/students/ipbc/31790", "max": "https://app.colaberry.com/app/ipbc/students/ipbc/40312", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StudentProfile"}, {"transformationsData": [{"feature_label": "ProjectName_2", "transformation_label": "String Indexer"}], "feature": "ProjectName_2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " AI Forge: Pathway to Predictive Excellence", "max": "The Effect of Holiday Sales on Walmart Using ML", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProjectName_2"}, {"transformationsData": [{"feature_label": "ProjectKPI_Link", "transformation_label": "String Indexer"}], "feature": "ProjectKPI_Link", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "https://app.colaberry.com/app/student/ProjMgmtByAdmin/1778", "max": "https://app.colaberry.com/app/student/ProjMgmtByAdmin/1956", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProjectKPI_Link"}]}))

	#transformationPostExecutionHook(workethicandproductivitycorrelationregressionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run workethicandproductivitycorrelationregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(workethicandproductivitycorrelationregressionautofe, ["internship_score2", "intern_mm_wkorder5", "WkScore", "UserID", "Dominance", "Influencer", "Steadiness", "Compliance", "Theoretical", "Utilitarian", "Aesthetic", "Social", "Individualistic", "Traditional", "Flexibility", "FuturisticThinking", "GoalOrientation", "InterpersonalSkills", "Leadership", "Negotiation", "PersonalResponsibility", "Persuasion", "PlanningOrganizing", "Presenting", "Teamwork", "WrittenCommunication", "UnderstandingOthers", "PracticalThinking", "SystemsJudgment", "SenseOfSelf", "RoleAwareness", "SelfDirection", "UnderstandingOthersBias", "ClassSignupsID", "SurveySent", "StagnantWks", "INTERNSHIP_SCORE52", "SurveysTaken", "SurveysGiven", "QuestionsScored", "Work Ethic", "Availability", "Team Player", "Quality of Work", "Trust_2", "Respectful", "SurveyScore", "BaseCamp_FinalScore", "IPBC_FinalScore", "BaseCamp_Score", "Attendance", "BaseCampCommentScore", "BaseCampTotalComments", "IPBC_Score", "InternID", "InternTechGroupID", "internisactive", "Intern_MM_WkOrder83", "Intern_MM_Wk", "Intern_MM_Yr", "Intern_MM_HW", "Intern_MM_Videos", "Intern_MM_VideosScore", "Intern_MM_AutoInt", "Intern_MM_Phone", "Intern_MM_LoginPer", "Intern_MM_ActivityScore", "Intern_MM_ParticipationPer", "Intern_MM_UserComments", "Intern_MM_ActivityScoreChk", "Intern_MM_UserCommentsChk", "Intern_MM_JRPChk", "Intern_MM_SubChk", "Intern_MM_AutoIntChk", "Intern_MM_PhoneChk", "Intern_MM_Wk_ActivityScore", "Intern_CC_Answer", "Intern_CC_CorrectAnswer", "Intern_CC_VideosHosted", "Intern_CC_QualfyVideo", "Intern_COE_Participation", "Intern_COE_ProjectUpdates", "IPBC_Part1", "IPBC_Part2", "IPBC_Part3", "IPBC_Score_Pre_Total", "InternLevelGroup", "projectname_stringindexer", "ProjectStartDate_dayofmonth", "ProjectStartDate_month", "ProjectStartDate_year", "Intern_stringindexer", "intern_mm_weekenddate_dayofmonth", "intern_mm_weekenddate_month", "intern_mm_weekenddate_year", "FirstName_stringindexer", "LastName_stringindexer", "Gender_stringindexer", "Email_stringindexer", "ReportDate_dayofmonth", "ReportDate_month", "ReportDate_year", "TopDISCscore_stringindexer", "LowDISCscore_stringindexer", "TopMotivator_stringindexer", "LowMotivator_stringindexer", "TopSkill_stringindexer", "LowSkill_stringindexer", "GoalResponseLen_stringindexer", "InternEmail_stringindexer", "InternStartDate_dayofmonth", "InternStartDate_month", "InternStartDate_year", "InternEndDate_stringindexer", "InternTechGroupName_stringindexer", "InternManager_stringindexer", "InternJobLocation_stringindexer", "InternBaseCampAlias_stringindexer", "PName_stringindexer", "L_URL_stringindexer", "Intern_LastActivitySection_stringindexer", "Intern_MM_Mentor_stringindexer", "Intern_MM_PlacementDate_stringindexer", "Intern_MM_WklyComments_stringindexer", "Intern_MM_LastCommentDate_stringindexer", "Phase_stringindexer", "CurrentSprintTask_stringindexer", "StudentProfile_stringindexer", "ProjectName_2_stringindexer", "ProjectKPI_Link_stringindexer"], "InternUserID")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

